In [1]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

import importlib

import RFSoC_Daq
from RFSoC_Daq import RFSoC_Daq

import Biquad
importlib.reload(Biquad)
from Biquad import Biquad

import SimBiquad
importlib.reload(SimBiquad)
from SimBiquad import SimBiquad

import Biquad_Daq
from Biquad_Daq import Biquad_Daq

In [2]:
daq = Biquad_Daq(None, None, 4, 2**10)
sim = SimBiquad()

Turning on SYNC
Turning off SYNC


In [3]:
import pynq

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [4]:
def update_plot(A, B, P, theta):
    daq.reset()
    
    daq.update_params(A, B, P, theta*np.pi)

    daq.run_Biquad()
    
    sim.update_params(A, B, P, theta*np.pi)
    sim.data = np.array(daq.adcBuffer[0] >> 4)
    
    # sim.set_daq_coeffs(daq.get_coeffs())
    
    sim.run_Biquad()
    
    daq_output = daq.extract_biquad()
    sim_output = sim.extract_biquad()
    daq_input = daq.extract_raw()
    
    fig, ((axB,axS,axR),(ax0, ax1, ax2)) = plt.subplots(2, 3, figsize=(35, 15))
        
    daq_output.plotWaveform(axB, title="Biquaded ADC224_T0_CH0")
    daq_output.plotFFT(ax0, title="Biquaded FFT")
    
    sim_output.plotWaveform(axS, title="Simulated Biquad")
    sim_output.plotFFT(ax1, title="Simulated FFT")
    
    daq_input.plotWaveform(axR, title="Raw ADC224_T0_CH0")
    daq_input.plotFFT(ax2, title="Raw FFT")
    
    plt.show()

In [5]:
A_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.05, value=0, description='A:', continuous_update=False)
B_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.05, value=1, description='B:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0, description='P:', continuous_update=False)
Theta_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.9601495285661642/np.pi, description='Theta:', continuous_update=False)

plot_output = widgets.Output()
    
def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        update_plot(A_slider.value, B_slider.value, P_slider.value, Theta_slider.value)
    
A_slider.observe(update, names='value')
B_slider.observe(update, names='value')
P_slider.observe(update, names='value')
Theta_slider.observe(update, names='value')

# Layout
slider_box = widgets.HBox([A_slider, B_slider, P_slider, Theta_slider])
ui = widgets.VBox([slider_box, plot_output])

# Display
display(ui)

# Initial plot
update(None)